In [1]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')

print(len(A))
print(len(B))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


16149
20238


In [2]:
A.head()

,name,address,zipcode,phone,cuisine,price,rating,review_count,ID
0,barn joo 35,"34 w 35th st, ,",10001,2.125644e+09,korean gastropubs tapas/small plates,$$,4.5,343,0
1,friedman's,"132 w 31st st, ,",10001,2.129719e+09,american (new) gluten-free breakfast & brunch,$$,4.0,1196,1
2,juniper,"237 w 35th st,",10001,2.129673e+09,cocktail bars american (new),$$,4.0,180,2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,6.464298e+09,gastropubs beer bar,$$,4.0,140,3
4,izakaya mew,"53 w 35th st, basement,",10001,6.463689e+09,sushi bars izakaya cocktail bars,$$,4.0,1232,4


## Blocking

In [3]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine', 'price'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine', 'violation_code', 'critical_flag', 'grade'])

In [4]:
print(len(C))

2859022


In [5]:
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=2)

0%                          100%
[##############################] | ETA: 00:01:09 | ETA: 00:01:02 | ETA: 00:00:56 | ETA: 00:00:52 | ETA: 00:00:49 | ETA: 00:00:47 | ETA: 00:00:45 | ETA: 00:00:43 | ETA: 00:00:41 | ETA: 00:00:39 | ETA: 00:00:37 | ETA: 00:00:35 | ETA: 00:00:33 | ETA: 00:00:31 | ETA: 00:00:29 | ETA: 00:00:27 | ETA: 00:00:25 | ETA: 00:00:23 | ETA: 00:00:21 | ETA: 00:00:19 | ETA: 00:00:17 | ETA: 00:00:15 | ETA: 00:00:13 | ETA: 00:00:11 | ETA: 00:00:09 | ETA: 00:00:07 | ETA: 00:00:05 | ETA: 00:00:03 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:56


In [6]:
# Describe the candidate set
print(len(D))
D.head(2)

216212


,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,ltable_price,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine,rtable_violation_code,rtable_critical_flag,rtable_grade
2,2,0,216,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,rocky's bar & restaurant,460 w 34th st,10001,pizza/italian,10B,Not Critical,A
8,8,0,340,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,naturopathica chelsea,127 w 26th st,10001,café/coffee/tea,10F,Not Critical,A


## Feature extraction

In [7]:
# Adding metadata to the files
em.set_key(A, 'ID')
em.set_key(B, 'ID')

em.set_key(D, '_id')
em.set_fk_ltable(D, 'ltable_ID')
em.set_fk_rtable(D, 'rtable_ID')
em.set_ltable(D, A)
em.set_rtable(D, B)

True

In [8]:
# Manual Feature Extraction
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)
features_to_drop = list(range(0, 32))
match_f = match_f.drop(features_to_drop)  # We drop all features, as we build features manually.

# Create blackbox features
import re
def custom_address_sim(ltuple, rtuple):
    sim = 0.0
    # Extract all numbers from the address and count how many of them match-
    numbers = re.findall(r'\d+', ltuple['address'])
    num_matches = 0
    for number in numbers:
        if number in rtuple['address']:
            num_matches = num_matches + 1
    if len(numbers) > 0:
        sim = float(num_matches) / float(len(numbers)) # Normalize    
    # Compute jaccard similarity for street names.
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['address']), em.tok_alphabetic(rtuple['address']))
    return sim * jaccard_sim

def custom_name_jaccard(ltuple, rtuple):
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return jaccard_sim

def custom_address_overlap(ltuple, rtuple):    
    sim = em.overlap_coeff(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_overlap(ltuple, rtuple):
    overlap_sim = em.overlap_coeff(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return overlap_sim

def custom_address_cosine(ltuple, rtuple):    
    sim = em.cosine(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_cosine(ltuple, rtuple):
    sim = em.cosine(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return sim

def custom_cuisine_overlap(ltuple, rtuple):
    sim = em.overlap_coeff(em.tok_alphabetic(ltuple['cuisine']), em.tok_alphabetic(rtuple['cuisine']))
    return sim

status = em.add_blackbox_feature(match_f, 'custom_address_sim', custom_address_sim)
status = em.add_blackbox_feature(match_f, 'custom_name_jaccard', custom_name_jaccard)
status = em.add_blackbox_feature(match_f, 'custom_address_overlap', custom_address_overlap)
status = em.add_blackbox_feature(match_f, 'custom_name_cosine', custom_name_cosine)
status = em.add_blackbox_feature(match_f, 'custom_address_cosine', custom_address_cosine)
status = em.add_blackbox_feature(match_f, 'custom_cuisine_overlap', custom_cuisine_overlap)

match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,custom_address_sim,None,None,None,None,None,<function custom_address_sim at 0x10e7430d0>,None,False
1,custom_name_jaccard,None,None,None,None,None,<function custom_name_jaccard at 0x10e7431e0>,None,False
2,custom_address_overlap,None,None,None,None,None,<function custom_address_overlap at 0x10e743268>,None,False
3,custom_name_cosine,None,None,None,None,None,<function custom_name_cosine at 0x10e743400>,None,False
4,custom_address_cosine,None,None,None,None,None,<function custom_address_cosine at 0x10e743378>,None,False
5,custom_cuisine_overlap,None,None,None,None,None,<function custom_cuisine_overlap at 0x10e743488>,None,False


In [9]:
D.head()

,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,ltable_price,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine,rtable_violation_code,rtable_critical_flag,rtable_grade
2,2,0,216,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,rocky's bar & restaurant,460 w 34th st,10001,pizza/italian,10B,Not Critical,A
8,8,0,340,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,naturopathica chelsea,127 w 26th st,10001,café/coffee/tea,10F,Not Critical,A
12,12,0,510,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,krush,2 w 32nd st,10001,american,08C,Not Critical,Z
14,14,0,583,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,market crates,26 w 33rd st,10001,delicatessen,02B,Critical,A
16,16,0,647,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,the kunjip,32 w 32nd st,10001,korean,02B,Critical,Z


In [10]:
H = em.extract_feature_vecs(D, feature_table=match_f, attrs_after=['ltable_name', 'ltable_address', 'ltable_zipcode', 'ltable_cuisine', 'ltable_price', 'rtable_name', 'rtable_address', 'rtable_zipcode', 'rtable_cuisine', 'rtable_violation_code', 'rtable_critical_flag', 'rtable_grade'])
H.head(10)


0%                          100%
[##############################] | ETA: 00:04:03 | ETA: 00:03:13 | ETA: 00:02:51 | ETA: 00:02:55 | ETA: 00:02:56 | ETA: 00:02:42 | ETA: 00:02:30 | ETA: 00:02:19 | ETA: 00:02:11 | ETA: 00:02:03 | ETA: 00:01:55 | ETA: 00:01:49 | ETA: 00:01:42 | ETA: 00:01:35 | ETA: 00:01:29 | ETA: 00:01:23 | ETA: 00:01:18 | ETA: 00:01:12 | ETA: 00:01:06 | ETA: 00:00:59 | ETA: 00:00:53 | ETA: 00:00:48 | ETA: 00:00:43 | ETA: 00:00:36 | ETA: 00:00:30 | ETA: 00:00:24 | ETA: 00:00:18 | ETA: 00:00:12 | ETA: 00:00:05 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:02:59


,_id,ltable_ID,rtable_ID,custom_address_sim,custom_name_jaccard,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,rtable_grade,...,rtable_violation_code,rtable_cuisine,rtable_zipcode,rtable_address,rtable_name,ltable_price,ltable_cuisine,ltable_zipcode,ltable_address,ltable_name
2,2,0,216,0.5,0.0,0.50,0.0,0.50,0.0,A,...,10B,pizza/italian,10001,460 w 34th st,rocky's bar & restaurant,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
8,8,0,340,0.0,0.0,0.50,0.0,0.50,0.0,A,...,10F,café/coffee/tea,10001,127 w 26th st,naturopathica chelsea,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
12,12,0,510,0.0,0.0,0.50,0.0,0.50,0.0,Z,...,08C,american,10001,2 w 32nd st,krush,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
14,14,0,583,0.0,0.0,0.50,0.0,0.50,0.0,A,...,02B,delicatessen,10001,26 w 33rd st,market crates,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
16,16,0,647,0.0,0.0,0.50,0.0,0.50,1.0,Z,...,02B,korean,10001,32 w 32nd st,the kunjip,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
17,17,0,656,0.0,0.0,0.50,0.0,0.50,1.0,Z,...,06E,korean,10001,2 w 32nd st,ddm yupdduk,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
21,21,0,700,0.5,0.0,0.50,0.0,0.50,0.0,A,...,04L,pizza,10001,151 w 34th st,pizza bar,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
22,22,0,758,0.5,0.0,0.75,0.0,0.75,0.0,A,...,06D,other,10001,224 w 35th st,2 brothers pizza,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
23,23,0,826,0.0,0.0,0.50,0.0,0.50,0.0,Z,...,06D,american,10001,251 w 30th st,slake,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35
24,24,0,844,0.0,0.0,0.50,0.0,0.50,0.0,Z,...,10F,café/coffee/tea,10001,59 w 30th st,madman espresso,$$,korean gastropubs tapas/small plates,10001,"34 w 35th st, ,",barn joo 35


In [12]:
print(len(H))

216212


## Matching

In [13]:
from sklearn.externals import joblib
matcher = joblib.load('../../stage-3/code/naive_bayes_trained.pkl')
# Predict on H
predictions = matcher.predict(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID','ltable_name', 'ltable_address', 'ltable_zipcode', 'ltable_cuisine', 'ltable_price', 'rtable_name', 'rtable_address', 'rtable_zipcode', 'rtable_cuisine', 'rtable_violation_code', 'rtable_critical_flag', 'rtable_grade'], 
              append=True, target_attr='predicted', inplace=False)
em.to_csv_metadata(predictions, '../datasets/matched_A_B.csv')

File already exists at ../datasets/matched_A_B.csv; Overwriting it
Metadata file already exists at ../datasets/matched_A_B.metadata. Overwriting it


True